In [ ]:
import random

# Lista de barcos
barcos = [
    {'nombre': 'portaaviones', 'tamano': 5},
    {'nombre': 'acorazado', 'tamano': 4},
    {'nombre': 'crucero', 'tamano': 3},
    {'nombre': 'submarino', 'tamano': 3},
    {'nombre': 'destructor', 'tamano': 2}
]

class Tablero:
    def __init__(self):
        """
        Inicializa el tablero de 10x10, representado por agua ('~') y crea una lista vacía de barcos.
        """
        self.tablero = [['~' for _ in range(10)] for _ in range(10)]
        self.barcos = []

    def colocar_barco(self, barco):
        """
        Coloca un barco en el tablero de manera aleatoria. Intenta colocar el barco con orientación
        horizontal o vertical en una posición disponible hasta que lo logra.
        """
        tamano = barco['tamano']
        while True:
            orientacion = random.choice(['H', 'V'])
            fila = random.randint(0, 9)
            columna = random.randint(0, 9)
            if self._puede_colocar_barco((fila, columna), orientacion, tamano):
                for i in range(tamano):
                    if orientacion == 'H':
                        self.tablero[fila][columna + i] = barco['nombre'][0].upper()
                    else:
                        self.tablero[fila + i][columna] = barco['nombre'][0].upper()
                posiciones = [(fila, columna + i) if orientacion == 'H' else (fila + i, columna) for i in range(tamano)]
                self.barcos.append({'nombre': barco['nombre'], 'tamano': tamano, 'posiciones': posiciones, 'impactos': 0})
                break

    def _puede_colocar_barco(self, posicion, orientacion, tamano):
        """
        Verifica si se puede colocar un barco en una posición dada sin que se solape con otros barcos o
        que se salga del límite del tablero.
        """
        fila, columna = posicion
        if orientacion == 'H' and columna + tamano > 10:
            return False
        if orientacion == 'V' and fila + tamano > 10:
            return False
        for i in range(tamano):
            if orientacion == 'H' and (columna + i >= 10 or self.tablero[fila][columna + i] != '~'):
                return False
            elif orientacion == 'V' and (fila + i >= 10 or self.tablero[fila + i][columna] != '~'):
                return False
        return True

    def colocar_barcos_automatico(self):
        """
        Coloca automáticamente todos los barcos definidos en el tablero llamando al método `colocar_barco`.
        """
        for barco in barcos:
            self.colocar_barco(barco)

    def recibir_ataque(self, fila, columna, jugador_nombre, mostrar=True, ocultar_barcos=False):
        """
        Procesa un ataque en una posición dada y muestra el resultado. Actualiza el tablero con 'O' si es agua,
        'X' si es un golpe a un barco. Si un barco se hunde, lo indica en el resultado.
        """
        if self.tablero[fila][columna] == '~':
            self.tablero[fila][columna] = 'O'
            resultado = "Agua"
        elif self.tablero[fila][columna] == 'O' or self.tablero[fila][columna] == 'X':
            resultado = "Ya atacado"
        else:
            self.tablero[fila][columna] = 'X'
            resultado = "Golpe"
            for barco in self.barcos:
                if (fila, columna) in barco['posiciones']:
                    barco['impactos'] += 1
                    if barco['impactos'] == barco['tamano']:
                        resultado = f"{barco['nombre']} hundido"
                    break

        # Muestra el tablero actualizado si `mostrar` es True
        if mostrar:
            print(f"\nTablero de {jugador_nombre} actualizado:")
            self.mostrar_tablero(ocultar_barcos=ocultar_barcos)
        return resultado

    def mostrar_tablero(self, ocultar_barcos=False):
        """
        Imprime el tablero en la consola, mostrando o no los barcos según el parámetro `ocultar_barcos`.
        """
        print("   1 2 3 4 5 6 7 8 9 10")
        fila_numero = 1
        for fila in self.tablero:
            fila_visual = [cell if cell in ('X', 'O') or not ocultar_barcos else '~' for cell in fila]
            print(f"{fila_numero:<2} " + ' '.join(fila_visual))
            fila_numero += 1

    def todos_barcos_hundidos(self):
        """
        Verifica si todos los barcos en el tablero han sido hundidos, retornando True o False.
        """
        return all(barco['impactos'] == barco['tamano'] for barco in self.barcos)

class Juego:
    """
    Inicializa el juego con dos tableros, nombres de jugadores, y coloca automáticamente los barcos en los tableros.
    """
    def __init__(self):
        self.tablero_jugador1 = Tablero()
        self.tablero_jugador2 = Tablero()
        self.nombre_jugador1 = "Jugador 1"
        self.nombre_jugador2 = "Computadora"
        self.tablero_jugador1.colocar_barcos_automatico()
        self.tablero_jugador2.colocar_barcos_automatico()
        self.jugador_actual = random.randint(1, 2)

    def atacar(self):
        """
        Pide al jugador ingresar la posición a atacar en el formato (fila, columna) y valida la entrada.
        """
        while True:
            try:
                entrada = input("Ingrese la posición a atacar (fila,columna): ")
                fila, columna = map(int, entrada.split(','))
                if 1 <= fila <= 10 and 1 <= columna <= 10:
                    return fila - 1, columna - 1
                else:
                    print("Error: Las coordenadas deben estar entre 1 y 10.")
            except ValueError:
                print("Error: Entrada inválida. Asegúrese de ingresar dos números separados por una coma.")

    def turno_jugador(self):
        """
        Realiza el turno del jugador humano. Muestra el resultado del ataque y verifica si el oponente
        tiene todos los barcos hundidos.
        """
        print("Jugador 1, elige la posición a atacar.")
        fila, columna = self.atacar()
        resultado = self.tablero_jugador2.recibir_ataque(fila, columna, self.nombre_jugador2, ocultar_barcos=True)
        print(resultado)
        return self.tablero_jugador2.todos_barcos_hundidos()

    def turno_computadora(self):
        """
        Realiza el turno de la computadora atacando aleatoriamente una posición. Muestra el resultado
        y verifica si el jugador tiene todos sus barcos hundidos.
        """
        print("\nTurno de Jugador 2 (Computadora).")
        fila, columna = random.randint(0, 9), random.randint(0, 9)
        print(f"La computadora elige atacar en: {fila + 1},{columna + 1}")
        resultado = self.tablero_jugador1.recibir_ataque(fila, columna, self.nombre_jugador1, ocultar_barcos=False)
        print(resultado)
        return self.tablero_jugador1.todos_barcos_hundidos()

    def iniciar_juego(self):
        """
        Controla el flujo del juego alternando entre los turnos del jugador y la computadora.
        Verifica si alguno de los dos ha ganado después de cada turno.
        """
        while True:
            if self.jugador_actual == 1:
                if self.turno_jugador():
                    print("¡Jugador 1 gana!")
                    break
            else:
                if self.turno_computadora():
                    print("¡Jugador 2 (Computadora) gana!")
                    break
            # Alterna el turno entre los jugadores
            self.jugador_actual = 2 if self.jugador_actual == 1 else 1

# Inicia el juego
if __name__ == "__main__":
    juego = Juego()
    juego.iniciar_juego()


Jugador 1, elige la posición a atacar.

Tablero de Computadora actualizado:
   1 2 3 4 5 6 7 8 9 10
1  O ~ ~ ~ ~ ~ ~ ~ ~ ~
2  ~ ~ ~ ~ ~ ~ ~ ~ ~ ~
3  ~ ~ ~ ~ ~ ~ ~ ~ ~ ~
4  ~ ~ ~ ~ ~ ~ ~ ~ ~ ~
5  ~ ~ ~ ~ ~ ~ ~ ~ ~ ~
6  ~ ~ ~ ~ ~ ~ ~ ~ ~ ~
7  ~ ~ ~ ~ ~ ~ ~ ~ ~ ~
8  ~ ~ ~ ~ ~ ~ ~ ~ ~ ~
9  ~ ~ ~ ~ ~ ~ ~ ~ ~ ~
10 ~ ~ ~ ~ ~ ~ ~ ~ ~ ~
Agua

Turno de Jugador 2 (Computadora).
La computadora elige atacar en: 2,6

Tablero de Jugador 1 actualizado:
   1 2 3 4 5 6 7 8 9 10
1  ~ ~ ~ ~ ~ ~ ~ ~ ~ ~
2  ~ S ~ ~ ~ O D D ~ ~
3  ~ S ~ ~ ~ P ~ ~ ~ ~
4  ~ S ~ ~ ~ P ~ ~ ~ ~
5  ~ ~ ~ ~ ~ P ~ ~ ~ ~
6  ~ ~ ~ ~ ~ P ~ C ~ ~
7  ~ ~ ~ ~ ~ P ~ C ~ ~
8  ~ ~ ~ ~ ~ ~ ~ C ~ ~
9  ~ ~ ~ ~ ~ ~ ~ ~ ~ ~
10 ~ ~ ~ ~ A A A A ~ ~
Agua
Jugador 1, elige la posición a atacar.

Tablero de Computadora actualizado:
   1 2 3 4 5 6 7 8 9 10
1  O ~ ~ ~ ~ ~ ~ ~ ~ ~
2  ~ O ~ ~ ~ ~ ~ ~ ~ ~
3  ~ ~ ~ ~ ~ ~ ~ ~ ~ ~
4  ~ ~ ~ ~ ~ ~ ~ ~ ~ ~
5  ~ ~ ~ ~ ~ ~ ~ ~ ~ ~
6  ~ ~ ~ ~ ~ ~ ~ ~ ~ ~
7  ~ ~ ~ ~ ~ ~ ~ ~ ~ ~
8  ~ ~ ~ ~ ~ ~ ~ ~ ~ ~
9  ~ ~ ~ ~ ~ ~ ~